# COGS 108 - EDA Checkpoint

# Names

- Naomi Chin
- Leo Friedman
- Jake Heinlein
- Dante Tanjuatco
- Nathan Tripp

<a id='research_question'></a>
# Research Question

*Is the combination of an mlb free agents age and batting performance, measured by relevant batting statistics (Batting Average, On Base Percentage, Slugging Average, etc.) indicative of their yearly salary, and if so, can we predict a player's future salary based on these factors?*

# Setup

In [50]:
# import packages and setup visuals
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import patsy
import statsmodels.api as sm
import statistics

# Data Cleaning

### Contracts

Concatenate all contract csv files from years 1991-2022 as a dataframe named "contracts"

In [51]:
# merge all contract files in contracts directory as one dataframe
directory = 'data/contracts/'
filepaths = [directory + filename for filename in os.listdir(directory)]
contracts = pd.concat([pd.read_csv(filepath) for filepath in filepaths])
contracts_original = contracts
print('contracts shape: ', contracts.shape)
print('contracts columns: ', list(contracts.columns))

contracts shape:  (4996, 18)
contracts columns:  ['Player', "Pos'n", 'Age', 'Qual    Offer', 'Old    Club', 'New Club', 'Years', 'Guarantee', 'Term', 'Option', 'Opt Out', 'AAV', 'Player Agent', 'Club Owner', 'Baseball Ops      head / club GM', 'Details', 'Age 7/1/21', 'Age 7/1/22']


Combine the three columns, 'Age', 'Age 7/1/21', and 'Age 7/1/22', into one 'Age' column by taking the non-null value from each set of columns (if it exists). There are 18 columns per contract observation. Drop all columns that are irrelevant to our project. Now that the dataframe only contains relevant variables, drop all observations with NaN values. 

In [52]:
# set 'Age' column to value in 'Age  7/1/21' column or 'Age 7/1/22' column if NaN
contracts['Age'] = contracts['Age'].add(contracts['Age 7/1/21'], fill_value=0)
contracts['Age'] = contracts['Age'].add(contracts['Age 7/1/22'], fill_value=0)

# drop unecessary columns
contracts = contracts[['Player','Pos\'n', 'Age', 'Term', 'AAV']]

# drop players with NaN values
contracts = contracts.dropna(axis=0)

Our project is looking at how batting performance affects salary. Pitcher's salaries, however, are not indicative of their batting performance. Drop all pitchers from the "contract" dataframe.

In [53]:
# drop players that are pitchers
contracts = contracts[contracts["Pos'n"].str.contains("hp") == False]

# drop position column (no longer needed)
contracts = contracts.drop('Pos\'n',axis=1)

display(contracts.head())

,Player,Age,Term,AAV
0,"Cano, Robinson",31.0,2014-23,"$24,000,000"
1,"Ellsbury, Jacoby",30.0,2014-20,"$21,857,143"
2,"Choo, Shin-Soo",31.0,2014-20,"$18,571,429"
3,"McCann, Brian",30.0,2014-18,"$17,000,000"
4,"Granderson, Curtis",33.0,2014-17,"$15,000,000"


Next, standardize our variables so we can plot and compare them with values from other datasets. 

In [54]:
# convert a players yearly salary (AAV) from a string to an int
def salary_to_int(str_in):
    return int(str_in.replace('$','').replace(',',''))

# convert the term of a player's contract (Term) to the year they were offered it
def term_to_year(str_in):
    return int(str(str_in).split('-')[0])

# standardize player names
def standardize_name(str_in):
    if ',' in str_in:
        name_list = str(str_in).split(',')
        str_in = name_list[1].strip() + " " + name_list[0].strip()
    str_in = str_in.replace('Jr.','')
    str_in = str_in.replace('Sr.','')
    str_in = str_in.replace('.','')
    str_in = str_in.replace('é','e')
    str_in = str_in.replace('á','a')
    return str_in.replace(' ', '')

# round all ages
def standardize_age(age):
    return round(float(age), 0)

In [55]:
# standardize salary and term variables
contracts['Player'] = contracts['Player'].apply(standardize_name)
contracts['Age'] = contracts['Age'].apply(standardize_age)
contracts['AAV'] = contracts['AAV'].apply(salary_to_int)
contracts['Term'] = contracts['Term'].apply(term_to_year)
display(contracts.head())

,Player,Age,Term,AAV
0,RobinsonCano,31.0,2014,24000000
1,JacobyEllsbury,30.0,2014,21857143
2,Shin-SooChoo,31.0,2014,18571429
3,BrianMcCann,30.0,2014,17000000
4,CurtisGranderson,33.0,2014,15000000


Rename contract columns for consistency. Drop all observations with contract values of $1. 

In [56]:
# rename columns for consistency
contracts.columns = ['playerName','playerAge','year','yearSalary']

# drop contracts that had a yearSalary of $1
contracts = contracts[contracts['yearSalary'] > 1]

display(contracts.head())

,playerName,playerAge,year,yearSalary
0,RobinsonCano,31.0,2014,24000000
1,JacobyEllsbury,30.0,2014,21857143
2,Shin-SooChoo,31.0,2014,18571429
3,BrianMcCann,30.0,2014,17000000
4,CurtisGranderson,33.0,2014,15000000


In [ ]:
# read people and batting files as dataframes
people = pd.read_csv('data/batting/People.csv')
batting = pd.read_csv('data/batting/Batting.csv')
batting_original = batting
print('people shape: ', people.shape)
print('batting shape: ', batting.shape)

In [ ]:
batting = batting[['playerID', 'yearID', 'AB', 'H', '2B', '3B', 'HR', 'BB','HBP','SF']]

In [ ]:
display(batting[batting['playerID'] == 'abramca01'])
batting = batting.groupby(['playerID', 'yearID']).agg(lambda x: x.sum(min_count=1))
batting = batting.reset_index()
display(batting[batting['playerID'] == 'abramca01'])

In [ ]:
# drop observations with NaN values
batting = batting.dropna(axis=0)

# drop observations with 0 At Bats
batting = batting[batting['AB'] > 0]
batting.shape

In [ ]:
# functions used for calculating batting average and OBS average
def calc_avg(h, ab):
    return h / ab
    
def calc_obp(h, bb, hbp, ab, sf):
    return (h + bb + hbp) / (ab + bb + sf + hbp)
    
def calc_tb(h, two_b, three_b, hr):
    singles = h - two_b - three_b - hr
    return singles + two_b * 2 + three_b * 3 + hr * 4

def calc_slg(tb, ab):
    return tb / ab

def calc_obs(obp, slg):
    return obp + slg

In [ ]:
# create Batting Average column in batting dataframe
avg = calc_avg(h=batting['H'], ab=batting['AB'])
batting['AVG'] = round(avg, 3)

# create On Base Percentage average column in batting dataframe
obp = calc_obp(h=batting['H'], bb=batting['BB'], hbp=batting['HBP'], ab=batting['AB'], sf=batting['SF'])
batting['OBP'] = round(obp, 3)

# create Slugging average column in batting dataframe
tb = calc_tb(h=batting['H'], two_b=batting['2B'], three_b=batting['3B'], hr=batting['HR'])
slg = calc_slg(tb, batting['AB'])
batting['SLG'] = round(slg, 3)

# create On Base Plus Slugging column in batting dataframe
obs = calc_obs(obp, slg)
batting['OBS'] = round(obs, 3)

batting.head()

In [ ]:
# drop irrelevant columns and rename
batting = batting[['playerID', 'yearID','AB', 'AVG', 'OBP','SLG','OBS']]
batting.columns = ['playerID', 'year', 'AB', 'AVG', 'OBP', 'SLG','OBS']

In [ ]:
# add name column to people that combines players first and last name
people['name'] = people['nameFirst'] + ' ' + people['nameLast']

# drop irrelevant columns
people = people[['playerID','name']]

# merge batting and people to add name column to batting dataframe
batting = batting.merge(people, how='left', on='playerID')

# rename and reorder  batting columns for consistency 
batting = batting.rename({'name':'playerName'}, axis=1)
batting = batting[['playerID','playerName','year','AB', 'AVG', 'OBP', 'SLG','OBS']]

# standardize playerName column
batting['playerName'] = batting['playerName'].apply(standardize_name)
batting.head()

In [ ]:
# remove players from batting who are not in contracts
batting = batting[batting['playerName'].isin(list(contracts['playerName'].unique()))]

# NOTE: 22 names in contracts missing from batting; Caused by naming disparities in databases;
print('number of missing names from batting: ', len(contracts['playerName'].unique()) - len(batting['playerName'].unique()))
print('batting: ', batting.shape)
print('cotracts: ', contracts.shape)

In [ ]:
print(len(contracts['playerName'].unique()))
print(len(batting['playerName'].unique()))

In [ ]:
# all names associated with a unique playerID
names = pd.Series([group[1]['playerName'].reset_index(drop=True)[0] for group in batting.groupby('playerID')])
name_counts = names.value_counts();

# list of players with the same name 
repeated_names = list(name_counts[name_counts > 1].index)

# remove batters and contracts with this name, I wish we didn't have to do this...
batting = batting[~batting['playerName'].isin(repeated_names)]
contracts = contracts[~contracts['playerName'].isin(repeated_names)]

In [ ]:
# results of cleaning: 
print('original contracts dataframe: ')
print('\tshape: ', contracts_original.shape)
print('\tunique players: ', len(list(contracts_original['Player'].unique())))

print('new contracts dataframe: ')
print('\tshape: ', contracts.shape)
print('\tunique players: ', len(list(contracts['playerName'].unique())),'\n')

print('original batting dataframe: ')
print('\tshape: ', batting_original.shape)
print('\tunique players: ', len(list(batting_original['playerID'].unique())))

print('new batting dataframe: ')
print('\tshape: ', batting.shape)
print('\tunique players: ', len(list(batting['playerName'].unique())))

# Data Analysis & Results (EDA)

In [ ]:
batting_grouped = batting.groupby(['playerName','year'])

In [ ]:
def getBattingStat(groupby, player, year, stat):
    try:
        return float(groupby.get_group((player, year))[stat])
    except: 
        return -1

In [ ]:
data = []
for index, contract in contracts.iterrows():
    if contract['playerName'] in list(batting['playerName']):
        year_salary = contract['yearSalary']
        stat = getBattingStat(batting_grouped, contract['playerName'], contract['year']-1, 'OBP')
        if stat > 0:
            data.append((stat, year_salary))
fig, ax = plt.subplots()
sns.scatterplot(data=pd.DataFrame(data=data, columns = ['OBP', 'year salary']), x='OBP', y='year salary')

Yearly salary predictions will depend on the year; factors including inflation and viewership are expected to alter MLB salaries from year to year. First, we will compare the MLB free agent contracts between 1991 and 2022.

In [ ]:
# compare yearly salaries between 1991 and 2022
fig = plt.figure(figsize=(15, 10))
plt.suptitle("Yearly Salaries", fontsize=14)
ax1 = fig.add_subplot(221)
ax2 = fig.add_subplot(222)
ax1.title.set_text('1991')
ax2.title.set_text('2022')
sns.histplot(data = contracts[contracts['year'] == 1991], x = 'yearSalary', bins = 30, ax = ax1)
sns.histplot(data = contracts[contracts['year'] == 2022], x = 'yearSalary', bins = 30, ax = ax2)
plt.show()

As expected, the average yearly salary jumped a significant amount from 1991 to 2022. The highest contract from 1991 is around $4,000,000 compared to the hihgest contract from 2022 being close to $35,000,000. 

We will now look at the average salary from each year of data that we have obtained (1991-2022).

In [ ]:
# compare average salaries over the years

salary_years = contracts['year'].value_counts().rename_axis('year').reset_index(name='counts')
salary_years['avg salary'] = ''
for i in range(len(salary_years)):
    salary_years['avg salary'][i] = statistics.mean(contracts[contracts['year'] == salary_years['year'][i]]['yearSalary'])
salary_years = salary_years.sort_values(by=['year']).reset_index(drop = True)
sns.scatterplot(data=salary_years, x='year', y='avg salary').set_title('Average Salary vs. Year')
plt.grid()
plt.show()
# try to fit the data (linear/exponential/etc)

The average salary has a general upward trend as the year increases. From these results, it is important to take into account the year when predicting salaries. 

In order to predict what information and stats are important for predicting salaries, we will take a look at the stats of the highest paid player from each year. Since we want to use previous performace to predict future salaries, we will first look at only the year prior to each contract. It will be helpful to get a better idea of what stats to expect given high salaries. 

In [ ]:
# look at the previous year stats for the highest contract from each year

# find the maximum salary from each year
salary_years['max salary'] = ''
for i in range(len(salary_years)):
    salary_years['max salary'][i] = ((contracts[contracts['year'] == salary_years['year'][i]]['yearSalary'])  
    [(contracts[contracts['year'] == salary_years['year'][i]]['yearSalary']).idxmax()])


# find players with max salary
max_player = (contracts[contracts['year']==1991][contracts[contracts['year']==1991]['yearSalary'] ==
 int(salary_years[salary_years['year']==1991]['max salary'])])

for i in range(len(salary_years)-1):
    max_player=max_player.append(contracts[contracts['year']==salary_years['year'][i+1]][contracts[contracts['year']==
    salary_years['year'][i+1]]['yearSalary'] == int(salary_years[salary_years['year']==salary_years['year'][i+1]]['max salary'])])
max_player = max_player.reset_index(drop = True)

# find the previous year stats for each max salary player
max_player['previous AVG'] = ''
max_player['previous AVG'][0] = np.nan
max_player['previous OBP'] = ''
max_player['previous OBP'][0] = np.nan
max_player['previous SLG'] = ''
max_player['previous SLG'][0] = np.nan
max_player['previous OBS'] = ''
max_player['previous OBS'][0] = np.nan
for i in range(len(max_player)-1):
    max_player['previous AVG'][i+1] = (float(batting[batting['year'] == max_player['year'][i]]
    [batting[batting['year'] == max_player['year'][i]]['playerName'] == max_player['playerName'][i]]['AVG']))
for i in range(len(max_player)-1):
    max_player['previous OBP'][i+1] = (float(batting[batting['year'] == max_player['year'][i]]
    [batting[batting['year'] == max_player['year'][i]]['playerName'] == max_player['playerName'][i]]['OBP']))
for i in range(len(max_player)-1):
    max_player['previous SLG'][i+1] = (float(batting[batting['year'] == max_player['year'][i]]
    [batting[batting['year'] == max_player['year'][i]]['playerName'] == max_player['playerName'][i]]['SLG']))
for i in range(len(max_player)-1):
    max_player['previous OBS'][i+1] = (float(batting[batting['year'] == max_player['year'][i]]
    [batting[batting['year'] == max_player['year'][i]]['playerName'] == max_player['playerName'][i]]['OBS']))


# find the minimum salary from each year
salary_years['min salary'] = ''
for i in range(len(salary_years)):
    salary_years['min salary'][i] = ((contracts[contracts['year'] == salary_years['year'][i]]['yearSalary'])  
    [(contracts[contracts['year'] == salary_years['year'][i]]['yearSalary']).idxmin()])


# can look at minimum salary stats similarly to how maximum stats were analyzed


Looking at the average salary, maximum salary, and minimum salary from the first three and last three years of data:

In [ ]:
print(salary_years.iloc[np.r_[:3, -3:0]])

The highest salary player from each year along with their previous year stats (AVG, OBP, SLG, OBS):

In [ ]:
print(max_player)